<a href="https://colab.research.google.com/github/songboy8/algonauts-GBM-2025/blob/main/gradientBoosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np

#Load up
data_path = "/content/drive/MyDrive/preds_all_subjs.npy"
data_raw = np.load(data_path, allow_pickle=True)
data = data_raw.item()
target_data_path = "/content/drive/MyDrive/all_true.npy"
target_data_raw = np.load(target_data_path, allow_pickle=True)
target_data = target_data_raw.item()
new_data = "/content/drive/MyDrive/algonauts/clip_pred.npy"
print(np.load(new_data, allow_pickle = True))

[[ 0.06737985  0.11345371  0.08428682 ... -0.06905887 -0.03962531
  -0.06076139]
 [ 0.06737985  0.11345371  0.08428682 ... -0.06905887 -0.03962531
  -0.06076139]
 [ 0.06737985  0.11345371  0.08428682 ... -0.06905887 -0.03962531
  -0.06076139]
 ...
 [ 0.02125257  0.05176201  0.05770828 ...  0.00311162 -0.01390609
   0.01350148]
 [ 0.01654082  0.03730971  0.02937785 ...  0.00797157 -0.02141222
   0.0092707 ]
 [ 0.02732364  0.05719461  0.07260095 ... -0.01103787 -0.00999358
   0.00193401]]


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#Concatenate 4 modalities into 1 subject for training
training_data = {}
for subject_id, modalities in data.items():
  conc_array = []
  for mod_name, array in modalities.items():
    if len(conc_array) == 0:
        conc_array = array
    else:
        conc_array = np.concatenate((conc_array, array), axis=1)
  training_data[subject_id] = conc_array
print(training_data)

{1: array([[ 0.05601881,  0.05541042,  0.11279497, ..., -0.0614173 ,
        -0.11584462, -0.08420394],
       [ 0.08187287,  0.10149819,  0.20398639, ..., -0.0614173 ,
        -0.11584462, -0.08420394],
       [ 0.09156215,  0.13222657,  0.25024638, ..., -0.0614173 ,
        -0.11584462, -0.08420394],
       ...,
       [ 0.06422111,  0.12657288,  0.1499795 , ..., -0.06505514,
         0.02682627, -0.01289973],
       [ 0.06422111,  0.12657288,  0.1499795 , ..., -0.04871498,
        -0.00188021, -0.01511706],
       [ 0.06422111,  0.12657288,  0.1499795 , ..., -0.02700329,
         0.0139455 ,  0.01496403]], dtype=float32), 2: array([[-0.11843091, -0.0866999 ,  0.03317614, ...,  0.12281027,
        -0.01769364,  0.01431469],
       [-0.13588521, -0.09623553,  0.063513  , ...,  0.12281027,
        -0.01769364,  0.01431469],
       [-0.11642025, -0.07016258,  0.08844066, ...,  0.12281027,
        -0.01769364,  0.01431469],
       ...,
       [ 0.05258396,  0.07569373,  0.09427509, ..., 

In [ ]:
#Time to train
final_preds = {}
for subject_id, X in training_data.items():
    y = target_data[subject_id]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    #make model
    bst = XGBRegressor()
    #train model
    bst.fit(X_train, y_train)
    #predict
    preds = bst.predict(X_test)
    print(f"{subject_id}'s Prediction: {preds}")
    final_preds[subject_id] = preds
np.save('aggregate_preds.npy',final_preds)